<a href="https://colab.research.google.com/github/maritnorli/IFCB_CNN_Classify/blob/main/PyTorch_Tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Have the required version but here is code that will install the latest version
try:
  import torch
  import torchvision
  assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
  assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 1.13+"
  print(f"torch version: {torch.__version__}")
  print(f"torchvision version: {torchvision.__version__}")
except:
  print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
  !pip3 install -U --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cull3
  import torch
  import torchvision
  print(f"torch version: {torch.__version__}")
  print(f"torchvision version: {torchvision.__version__}")


[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cull3
torch version: 2.2.2+cu121
torchvision version: 0.17.2+cu121


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## Import the colab libraries and set up paths
from google.colab import drive
import os

## copy files from drive to colab
import shutil
# Specify the path to your "going_modular" folder in Google Drive
drive_path = "/content/drive/MyDrive/IFCB/going_modular"


# List files in the "going_modular" folder
files = os.listdir(drive_path)

# Copy each file individually to the /content directory
for file in files:
    src = os.path.join(drive_path, file)
    dst = os.path.join("/content", file)
    try:
        shutil.copy(src, dst)
    except Exception as e:
        print(f"Failed to copy {file}: {e}")

# wait a minute or so before it pops up

In [4]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
# Continue with regular imports
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Now try importing modules directly from the copied drive files
try:
    import data_setup
    import engine

except ModuleNotFoundError as e:
    print(f"Failed to import modules: {e}")

[INFO] Couldn't find torchinfo... installing it.


In [8]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)


2.2.2+cu121
0.17.2+cu121


In [9]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [10]:
set_seeds()

In [12]:
import os
#import zipfile will likely be usefull later
from pathlib import Path

# Set data paths
data_path = Path("/content/drive/MyDrive/IFCB/data/")
image_path = Path("/content/drive/MyDrive/IFCB/data/IFCB_test_train")

# Setup Dirs
train_dir = image_path / "Train"
test_dir = image_path / "Test"

# Setup ImageNet normalization levels
# See here: https://pytorch.org/vision/0.12/models.html
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Create transform pipeline manually
from torchvision import transforms
manual_transforms = transforms.Compose([
                                        transforms.Resize((224, 224)),
                                        transforms.Grayscale(num_output_channels=3),
                                        transforms.ToTensor(),
                                        normalize
])
print(f"Manually created transforms: {manual_transforms}")

# Create DataLoaders

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform= manual_transforms, # resize, convert images to between 0 & 1 and normalize them
                                                                               batch_size=32) # set mini-batch size to 32

train_dataloader, test_dataloader, class_names

#### Make an EfficientNet_V2_S model

In [14]:
import torchvision.models as models
effnetv2_s_weight = models.EfficientNet_V2_S_Weights.DEFAULT #Default best available weights
effnetv2_s = models.efficientnet_v2_s(weights=effnetv2_s_weight)
#effnetv2_s #First run this then get the base layer:
#    (0): Dropout(p=0.2, inplace=True)
#    (1): Linear(in_features=1280, out_features=1000, bias=True)

In [15]:
from torchinfo import summary
summary(model=effnetv2_s,
        input_size=(1,3,244,244))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [1, 1000]                 --
├─Sequential: 1-1                                       [1, 1280, 8, 8]           --
│    └─Conv2dNormActivation: 2-1                        [1, 24, 122, 122]         --
│    │    └─Conv2d: 3-1                                 [1, 24, 122, 122]         648
│    │    └─BatchNorm2d: 3-2                            [1, 24, 122, 122]         48
│    │    └─SiLU: 3-3                                   [1, 24, 122, 122]         --
│    └─Sequential: 2-2                                  [1, 24, 122, 122]         --
│    │    └─FusedMBConv: 3-4                            [1, 24, 122, 122]         5,232
│    │    └─FusedMBConv: 3-5                            [1, 24, 122, 122]         5,232
│    └─Sequential: 2-3                                  [1, 48, 61, 61]           --
│    │    └─FusedMBConv: 3-6                         